# DS-636 Final Project :  Predict Future Sales
## Project Team Members
#### Mallika Kasi mk2434
#### Lizbeth Fernandez lf243

In [46]:
###Loading the Libraries
install.packages("xgboost")
library(xgboost)
library(dplyr)
library(data.table)
library(caret)

Warning message:
"package 'xgboost' is in use and will not be installed"


In [3]:
###Loading the DataFiles
item_categeries = read.csv(file ="item_categories.csv")
items = read.csv(file ="items.csv")
sales_train = read.csv(file="sales_train.csv")
sample_submission = read.csv(file="sample_submission.csv")
shops = read.csv(file="shops.csv")
test = read.csv(file ="test.csv")

In [4]:
head(item_categeries)
head(items)
head(sales_train)
head(sample_submission)
head(shops)
head(test)

,item_category_name,item_category_id
,<chr>,<int>
1,PC - Гарнитуры/Наушники,0
2,Аксессуары - PS2,1
3,Аксессуары - PS3,2
4,Аксессуары - PS4,3
5,Аксессуары - PSP,4
6,Аксессуары - PSVita,5


,item_name,item_id,item_category_id
,<chr>,<int>,<int>
1,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
2,"!ABBYY FineReader 12 Professional Edition Full [PC, Цифровая версия]",1,76
3,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
4,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
5,***КОРОБКА (СТЕКЛО) D,4,40
6,***НОВЫЕ АМЕРИКАНСКИЕ ГРАФФИТИ (UNI) D,5,40


,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
,<chr>,<int>,<int>,<int>,<dbl>,<dbl>
1,02.01.2013,0,59,22154,999.00,1
2,03.01.2013,0,25,2552,899.00,1
3,05.01.2013,0,25,2552,899.00,-1
4,06.01.2013,0,25,2554,1709.05,1
5,15.01.2013,0,25,2555,1099.00,1
6,10.01.2013,0,25,2564,349.00,1


,ID,item_cnt_month
,<int>,<dbl>
1,0,0.5
2,1,0.5
3,2,0.5
4,3,0.5
5,4,0.5
6,5,0.5


,shop_name,shop_id
,<chr>,<int>
1,"!Якутск Орджоникидзе, 56 фран",0
2,"!Якутск ТЦ ""Центральный"" фран",1
3,"Адыгея ТЦ ""Мега""",2
4,"Балашиха ТРК ""Октябрь-Киномир""",3
5,"Волжский ТЦ ""Волга Молл""",4
6,"Вологда ТРЦ ""Мармелад""",5


,ID,shop_id,item_id
,<int>,<int>,<int>
1,0,5,5037
2,1,5,5320
3,2,5,5233
4,3,5,5232
5,4,5,5268
6,5,5,5039


In [5]:
### Checking Train Data for null values
X=apply(sales_train,2, function (x) any (is.na(x)))
### Checking Test Data for null values
Y=apply(test,2, function (x) any (is.na(x)))  

X
Y        

date date_block_num        shop_id        item_id     item_price 
         FALSE          FALSE          FALSE          FALSE          FALSE 
  item_cnt_day 
         FALSE

ID shop_id item_id 
  FALSE   FALSE   FALSE

In [7]:
#### Merge sales_train data with items, item categories, and shops

salesMergeDF = sales_train %>%
  left_join(items, by = "item_id") %>%
  left_join(item_categeries, by = "item_category_id") %>%
  left_join(shops, by = "shop_id")

head(salesMergeDF)

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_name,item_category_id,item_category_name,shop_name
,<chr>,<int>,<int>,<int>,<dbl>,<dbl>,<chr>,<int>,<chr>,<chr>
1,02.01.2013,0,59,22154,999.00,1,ЯВЛЕНИЕ 2012 (BD),37,Кино - Blu-Ray,"Ярославль ТЦ ""Альтаир"""
2,03.01.2013,0,25,2552,899.00,1,DEEP PURPLE The House Of Blue Light LP,58,Музыка - Винил,"Москва ТРК ""Атриум"""
3,05.01.2013,0,25,2552,899.00,-1,DEEP PURPLE The House Of Blue Light LP,58,Музыка - Винил,"Москва ТРК ""Атриум"""
4,06.01.2013,0,25,2554,1709.05,1,DEEP PURPLE Who Do You Think We Are LP,58,Музыка - Винил,"Москва ТРК ""Атриум"""
5,15.01.2013,0,25,2555,1099.00,1,DEEP PURPLE 30 Very Best Of 2CD (Фирм.),56,Музыка - CD фирменного производства,"Москва ТРК ""Атриум"""
6,10.01.2013,0,25,2564,349.00,1,DEEP PURPLE Perihelion: Live In Concert DVD (Кир.),59,Музыка - Музыкальное видео,"Москва ТРК ""Атриум"""


In [9]:
### Aggregate sales_train Datas to monthly level
MonthlySalesDF = salesMergeDF %>% group_by(date_block_num,shop_id,item_id) %>% summarize(TotalSaleMonthlyCnt = sum(item_cnt_day) ,  .groups = 'drop') 

head(MonthlySalesDF)
dim(MonthlySalesDF)

date_block_num,shop_id,item_id,TotalSaleMonthlyCnt
<int>,<int>,<int>,<dbl>
0,0,32,6
0,0,33,3
0,0,35,1
0,0,43,1
0,0,51,2
0,0,61,1


[1] 1609124       4

In [10]:
### Adding lag features ( lag features allow the model to use sales patterns from these months to predict sales for November 2015 )
for (i in 1:12) {
  MonthlySalesDF = MonthlySalesDF %>%
    group_by(shop_id, item_id) %>%
    mutate(!!paste0("lag_", i) := lag(TotalSaleMonthlyCnt, n = i, order_by = date_block_num))
}

head(MonthlySalesDF)
dim(MonthlySalesDF)

date_block_num,shop_id,item_id,TotalSaleMonthlyCnt,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,lag_8,lag_9,lag_10,lag_11,lag_12
<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0,0,32,6,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
0,0,33,3,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
0,0,35,1,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
0,0,43,1,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
0,0,51,2,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
0,0,61,1,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


[1] 1609124      16

In [11]:
# Fill NAs with 0s for all columns in MonthlySalesDF
MonthlySalesDF[is.na(MonthlySalesDF)] = 0
MonthlySalesDF

date_block_num,shop_id,item_id,TotalSaleMonthlyCnt,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,lag_8,lag_9,lag_10,lag_11,lag_12
<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0,0,32,6,0,0,0,0,0,0,0,0,0,0,0,0
0,0,33,3,0,0,0,0,0,0,0,0,0,0,0,0
0,0,35,1,0,0,0,0,0,0,0,0,0,0,0,0
0,0,43,1,0,0,0,0,0,0,0,0,0,0,0,0
0,0,51,2,0,0,0,0,0,0,0,0,0,0,0,0
0,0,61,1,0,0,0,0,0,0,0,0,0,0,0,0
0,0,75,1,0,0,0,0,0,0,0,0,0,0,0,0
0,0,88,1,0,0,0,0,0,0,0,0,0,0,0,0
0,0,95,1,0,0,0,0,0,0,0,0,0,0,0,0


### Training Using Linear Regression Model


In [30]:
### prediction Using Linear Regression Model
# Split data into training and validation sets
set.seed(123)
train_index = createDataPartition(MonthlySalesDF$TotalSaleMonthlyCnt, p = 0.8, list = FALSE)
train_data = MonthlySalesDF[train_index, ]
validation_data = MonthlySalesDF[-train_index, ]
# Select features for linear regression
#features = c("shop_id", "item_id", "item_category_id")  # Add more features as needed
train_data = train_data %>% select(shop_id, item_id,TotalSaleMonthlyCnt)
validation_data = validation_data %>% select(shop_id, item_id,TotalSaleMonthlyCnt)

# Convert categorical variables to factors
train_data = train_data %>% mutate(across(where(is.character), as.factor))
validation_data = validation_data %>% mutate(across(where(is.character), as.factor))
# Build the linear regression model
model = lm(TotalSaleMonthlyCnt ~ ., data = train_data)

# Summary of the model
#summary(model)
# Make predictions on the validation set
predictions = predict(model, newdata = validation_data)

# Calculate RMSE
rmse = sqrt(mean((validation_data$TotalSaleMonthlyCnt - predictions)^2))
print(paste("Validation RMSE Using Linear Regression Model:", rmse))

[1] "Validation RMSE Using Linear Regression Model: 8.42755344462933"


### Prediction Using Linear Regression Model

In [55]:
### Prepare test data and Predict the sale for November 2015 for the Data Provided in the test file
TestMerge = test %>% select(shop_id, item_id, TotalSaleMonthlyCnt = 0) 

TestMerge = TestMerge %>% mutate(across(where(is.character), as.factor))

predictions_LM = predict(model, newdata = TestMerge)
### Prepare Output file
submission_LM = data.frame(ID = test$ID, TotalSaleMonthlyCnt = round(predictions_LM,1))
submission_LM

write.csv(submission_LM, 'submission.csv', row.names = FALSE)


,ID,TotalSaleMonthlyCnt
,<int>,<dbl>
1,0,2.3
2,1,2.3
3,2,2.3
4,3,2.3
5,4,2.3
6,5,2.3
7,6,2.3
8,7,2.3
9,8,2.3


### Training Using XG Bosst Model


In [31]:
### prediction Using XG Boost Model
### Split the SalesTrain data for training and Validation
Train_data = MonthlySalesDF %>%  filter(date_block_num < 33)
Validate_data = MonthlySalesDF %>%  filter(date_block_num == 33)

dim(Train_data)
dim(Validate_data)

[1] 1577593      16

[1] 31531    16

In [32]:
### Remove the target variable from the features
Train_data_Features = Train_data %>%  select(-TotalSaleMonthlyCnt)
Validate_data_Features = Validate_data %>%  select(-TotalSaleMonthlyCnt)

### Convert to matrix
TrainDataMatrix = as.matrix(Train_data_Features)
ValidateDataMatrix = as.matrix(Validate_data_Features)

### Create DMatrix for XGBoost
TrainDataMatrix = xgb.DMatrix(data = TrainDataMatrix, label = Train_data$TotalSaleMonthlyCnt)
ValidateDatamatrix = xgb.DMatrix(data = ValidateDataMatrix, label = Validate_data$TotalSaleMonthlyCnt)

TrainDataMatrix
ValidateDatamatrix

xgb.DMatrix  dim: 1577593 x 15  info: label  colnames: yes

xgb.DMatrix  dim: 31531 x 15  info: label  colnames: yes

In [36]:
### Train XGBoost model
params = list(objective = "reg:squarederror", eval_metric = "rmse", eta = 0.1, max_depth = 6)
xgb_model = xgboost(data = TrainDataMatrix, params = params, nrounds = 100)

[1]	train-rmse:8.192050 
[2]	train-rmse:7.778298 
[3]	train-rmse:7.421001 
[4]	train-rmse:7.113111 
[5]	train-rmse:6.849558 
[6]	train-rmse:6.599166 
[7]	train-rmse:6.402457 
[8]	train-rmse:6.213492 
[9]	train-rmse:6.061981 
[10]	train-rmse:5.932777 
[11]	train-rmse:5.813458 
[12]	train-rmse:5.720813 
[13]	train-rmse:5.636126 
[14]	train-rmse:5.565941 
[15]	train-rmse:5.506076 
[16]	train-rmse:5.448068 
[17]	train-rmse:5.402296 
[18]	train-rmse:5.348713 
[19]	train-rmse:5.314354 
[20]	train-rmse:5.284043 
[21]	train-rmse:5.254484 
[22]	train-rmse:5.233387 
[23]	train-rmse:5.211320 
[24]	train-rmse:5.159750 
[25]	train-rmse:5.144023 
[26]	train-rmse:5.129133 
[27]	train-rmse:5.113976 
[28]	train-rmse:5.101224 
[29]	train-rmse:5.065421 
[30]	train-rmse:5.054126 
[31]	train-rmse:5.038775 
[32]	train-rmse:5.031401 
[33]	train-rmse:5.019625 
[34]	train-rmse:5.012573 
[35]	train-rmse:5.001857 
[36]	train-rmse:4.996312 
[37]	train-rmse:4.990540 
[38]	train-rmse:4.987175 
[39]	train-rmse:4.982

### Prediction Using XG Boost Model


In [37]:
### Predictions for Validate Data
Validate_predictions = predict(xgb_model, ValidateDatamatrix)

### Calculate RMSE
rmse = sqrt(mean((Validate_predictions - Validate_data$TotalSaleMonthlyCnt)^2))
print(paste("Validation RMSE Using XG Boost Model:", rmse))

[1] "Validation RMSE Using XG Boost Model: 12.8157079287564"


In [39]:
### Prepare test data
TestMerge_XG = test %>% mutate(date_block_num = 34 , TotalSaleMonthlyCnt = 0) 
### Adding lag features for Test Data( lag features allow the model to use sales patterns from these months to predict sales for November 2015 )
for (i in 1:12) {
    str1  = paste0("lag_", i)
   TestMerge_XG = TestMerge_XG %>%   mutate(!!str1 := 0)
}

Test_data_Features = TestMerge_XG %>% select(date_block_num,shop_id,item_id,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,lag_8,lag_9,lag_10,lag_11,lag_12)
#head(Test_data_Features)
### Convert to matrix
TestMatrix = as.matrix(Test_data_Features)


train_features = colnames(TrainDataMatrix)
test_features = colnames(TestMatrix)
train_features
test_features
### Prepare test matrix
TestMatrix = xgb.DMatrix(data = TestMatrix, label = TestMerge_XG$TotalSaleMonthlyCnt)

### Predict sales for November 2015
Test_predictions_XG = predict(xgb_model, TestMatrix)

### Prepare Output file
submission_XG = data.frame(ID = test$ID, TotalSaleMonthlyCnt = Test_predictions_XG)
submission_XG

[1] "date_block_num" "shop_id"        "item_id"        "lag_1"         
 [5] "lag_2"          "lag_3"          "lag_4"          "lag_5"         
 [9] "lag_6"          "lag_7"          "lag_8"          "lag_9"         
[13] "lag_10"         "lag_11"         "lag_12"

[1] "date_block_num" "shop_id"        "item_id"        "lag_1"         
 [5] "lag_2"          "lag_3"          "lag_4"          "lag_5"         
 [9] "lag_6"          "lag_7"          "lag_8"          "lag_9"         
[13] "lag_10"         "lag_11"         "lag_12"

ID,TotalSaleMonthlyCnt
<int>,<dbl>
0,2.75658
1,2.75658
2,2.75658
3,2.75658
4,2.75658
5,2.75658
6,2.75658
7,2.75658
8,2.75658


### Prediction Sample Output Using Both Models


In [41]:
###Submission File output Sample using Linear Modelling
head(submission_LM)
tail(submission_LM)
###Submission File output Sample using XG Boost Modelling
head(submission_XG)
tail(submission_XG)

,ID,TotalSaleMonthlyCnt
,<int>,<dbl>
1,0,2.347264
2,1,2.343076
3,2,2.344364
4,3,2.344378
5,4,2.343846
6,5,2.347234


,ID,TotalSaleMonthlyCnt
,<int>,<dbl>
214195,214194,2.159826
214196,214195,2.153508
214197,214196,2.187035
214198,214197,2.193411
214199,214198,2.135842
214200,214199,2.412206


,ID,TotalSaleMonthlyCnt
,<int>,<dbl>
1,0,2.75658
2,1,2.75658
3,2,2.75658
4,3,2.75658
5,4,2.75658
6,5,2.75658


,ID,TotalSaleMonthlyCnt
,<int>,<dbl>
214195,214194,1.869999
214196,214195,1.869999
214197,214196,1.910695
214198,214197,1.910695
214199,214198,1.869999
214200,214199,1.871673


### Conclusion:
#####  The Root Mean Square values are calculated using Linear Regression and XG Boost Models.
######      Validation RMSE Using Linear Regression Model: 8.42755344462933
######      Validation RMSE Using XG Boost Model: 12.8157079287564
######  Forecasted the total number of products sold in every shop for the given test set for November 2015 using both Methods.
######  Since the Linear Regression Model's RMSE is Lower than the XG Boost Model, this confirms the Linear Regression Model's Prediction is better than the XB Boost, So submitting the Predicted values using the Linear Regression Model.
